# **Tell Me What You Want, I'll Tell You What You Like**

In this file, we try to use an approach using Neural Network 

## DOcplex Python API installation

First, we install the docplex python API which we will use for modelling our problem and solving it. (Then, we will not have problem size error)

In [ ]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        !pip install docplex -q
        print("!pip install docplex -q...")
    else:
        !pip install --user docplex -q
        print("!pip install --user docplex -q...")
        
print("End of the intallation of python API...")

## Main code

In [ ]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [ ]:
#############################
### Essentials functions ###
#############################
import FunctionMain as fm
import Function_NN as nn

In [ ]:
# ----------------- Parameters

# The file to consider
file = './file_with_optimal_val/la04.txt'

# The number of solutions that we will have in the solver during the first iteration
k = 10

# The number of layer which is fixed
nb_layers = 5

# Variable which display the solution
display_sol = False

# Variable which display the start (in a vector)
display_start = False

# Variable which display the start (in a matrix)
display_matrix = False

# Time stop criterion 
tps_max = 300

# Number of iteration stop criterion
it_max = 20

# Parameters of the first neural network
nb_hidden_layers1 = 1
nb_neurons1 = [3, 1]

# Parameters of the second neural network
nb_hidden_layers2 = 1
nb_neurons2 = [3, 1]


# The number of solutions that we will have in the solver after the first iteration
k_k = 10

In [ ]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# --------- Interaction with the solver
data = []
n, m, data, T_machine, T_duration, duration, optimalval = fm.get_data_from_file(file)

# --------- Call Solver constructor in Solver.py and create the variables of the model
model, solver, tasks = fm.initialize_solver(data, n, m, duration)

model, variables = solver.create_constraints(model, n, m, optimalval, T_machine)

# ------------ Solve the model
print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)

# ------------ Display the result
# fm.display_solution(msol, display_sol)
print("Model solved !")

# ---------------- Interaction with the user
print("\n--------Interaction with the user...---------")
print("\nCreating the user...")
user = User()
print("User created !")

#Get the variables of the model

list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)

# Vector of the start time of each task of each preference
starts = user.start_pref(n, m, display_start)

# Matrix of the start time of each task of each preference
matrix = user.matrix_pref(n, m, display_matrix)

# Testing the order of preferences and the differences between solutions
fm.test(n, m, user)

print("list layers : ",list_layers)

sol_layers = fm.list_list_list_start_of_tasks(n, m, list_layers)

In [ ]:
####################################################################
#### NOUVELLES SOLUTIONS ET NOUVEAUX CLUSTERING A CHAQUE ITERATION
####################################################################

###  -------------- Iteration of the solver with the preferences
it = 1
tps = runtime
list_min_obj = [min(list_obj)]
list_min_obj_global = [min(list_obj)]

criterion = (tps < tps_max) and (it < it_max) 

# ----------------- Add the preferences to the model
while criterion :
    print("\n--------Iteration {}---------".format(it))
    it += 1

    # --------- Call Solver constructor in Solver.py and create the variables of the model
    model, solver, tasks = fm.initialize_solver(data, n, m, duration)

    # --------- Add the new constraints to the model (that solution must be different from the previous generated solutions)
    variables = fm.update_variables_new_constraint(n, m,  pref, model, solver)
    
    model, variables = solver.create_constraints(model, n, m, optimalval, T_machine)


    # ---------------- Interaction with the neural network

    weights1s = nn.find_perfect_NN(data, n, m, sol_layers, nb_hidden_layers1, nb_neurons1)

    weights2 = nn.find_perfect_NN(data, n, m, sol_layers, nb_hidden_layers2, nb_neurons2)

    print(weights1.get_solution())

    # Are the neural networks perfect ?
    accuracy1 = nn.accurate_NN(model, solver, sol_layers, weights1, nb_hidden_layers1, nb_neurons1)
    accuracy2 = nn.accurate_NN(model, solver, sol_layers, weights2, nb_hidden_layers2, nb_neurons2)

    while (accuracy2 and accuracy1) == False:

        print("Update the neural network...")

        if not accuracy1:
            nb_neurons1[nb_hidden_layers1-1]+=1
            weights1 = nn.find_perfect_NN(data, n, m, sol_layers, nb_hidden_layers1, nb_neurons1)
            solver.add_constraint(model, weights1 != weights2)
            accuracy1 = nn.accurate_NN(model, solver, sol_layers, weights1, nb_hidden_layers1, nb_neurons1)

        if not accuracy2:
            nb_neurons2[nb_hidden_layers1-1]+=1
            weights2 = nn.find_perfect_NN(data, n, m, sol_layers, nb_hidden_layers2, nb_neurons2)
            solver.add_constraint(model, weights1 != weights2)
            accuracy2 = nn.accurate_NN(model, solver, sol_layers, weights2, nb_hidden_layers2, nb_neurons2)

    

    # Compare the result of 2 different neural networks for the same solution

    tasks_starts = []
    for i in range(n):
        for j in range(m):
            tasks_starts.append(model.start_of(variables[i][j]))

    outputvar_NN1 = nn.activation_function(model, solver, tasks_starts, weights1, nb_hidden_layers1, nb_neurons1)
    outputvar_NN2 = nn.activation_function(model, solver, tasks_starts, weights2, nb_hidden_layers2, nb_neurons2)
    solver.add_constraint(model, outputvar_NN1 != outputvar_NN2)
    # solver.add_constraint(model, logical_or(outputvar_NN1 == 1, outputvar_NN2 == 1))


    # ------------ Solve the model
    print("\nSolving the model...")
    msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)
    print("The number of solutions generated is :",nb_solution)
    # list = []
    # for sol in msol:
    #     list.append(user.objectiveFunction(sol)*user.objectiveFunctionRegularity(sol, n, m))

    # list_min_obj.append(min(list))
    # print("Objective function :", list_min_obj)

    # ------------ Display the result
    fm.display_solution(msol, display_sol)
    print("Model solved !")

    # ---------------- Interaction with the user
    list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)
    print("Il y a {} solution(s)".format(len(pref)))

    sol_layers = fm.list_list_list_start_of_tasks(n, m, list_layers)

    # list_min_obj_global.append(min(list_obj))
    # print("Objective function global :", list_min_obj_global)

    # Vector of the start time of each task of each preference
    starts = user.start_pref(n, m, display_start)

    # Matrix of the start time of each task of each preference
    matrix = user.matrix_pref(n, m, display_matrix)

    # Testing the order of preferences and the differences between solutions
    fm.test(n, m, user)

#------------------ Condition d'arrêt ------------------
    tps += runtime
    criterion = (tps < tps_max) and (it < it_max)
    fm.stopCondition(it, it_max, tps, tps_max)